In [84]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import random
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

np.set_printoptions(suppress=True)

In [85]:
header  = ["Click", "Weekday", "Hour", "Timestamp", "Log Type", "User ID", "User‐Agent", "IP", "Region", "City", "Ad Exchange", "Domain", "URL", "Anonymous URL ID", "Ad slot ID", "Ad slot width", "Ad slot height", "Ad slot visibility", "Ad slot format", "Ad slot floor price (RMB/CPM)", "Creative ID", "Key Page URL", "Advertiser ID", "User Tags"]
print(header)

['Click', 'Weekday', 'Hour', 'Timestamp', 'Log Type', 'User ID', 'User\xe2\x80\x90Agent', 'IP', 'Region', 'City', 'Ad Exchange', 'Domain', 'URL', 'Anonymous URL ID', 'Ad slot ID', 'Ad slot width', 'Ad slot height', 'Ad slot visibility', 'Ad slot format', 'Ad slot floor price (RMB/CPM)', 'Creative ID', 'Key Page URL', 'Advertiser ID', 'User Tags']


In [86]:
trainfile = 'train_sample.txt'
testfile = 'test.txt'

train = pd.read_csv(trainfile, header = None, sep = '\t', names = header)
test = pd.read_csv(testfile, header = None, sep = '\t', names = header[1:])

trainlen = len(train)
testlen = len(test)

### Show head lines of files
print(train.head())

   Click  Weekday  Hour          Timestamp  Log Type          User ID  \
0      0        6     0  20130606000104589         1  VhkS1DK53UjsBVL   
1      0        6     0  20130606000104622         1  VhL0O5FD32m63hl   
2      0        6     0  20130606000104809         1  Vhk7ZAnxD9lfQoL   
3      0        6     0  20130606000104878         1  Vhk7ZAnx3Tmdjda   
4      0        6     0  20130606000104991         1  Vhk7ZAnx3cB9tbc   

   User‐Agent             IP  Region  City  \
0  windows_ie  221.228.142.*      80    82   
1  windows_ie     221.10.5.*     276   277   
2  windows_ie    60.160.94.*     308   321   
3  windows_ie    210.21.84.*     216   217   
4  windows_ie  222.243.167.*     201   204   

                  ...                                     Ad slot ID  \
0                 ...                    mm_14539978_2071324_8355258   
1                 ...                   mm_24597501_3474831_11374379   
2                 ...                    mm_27762412_2408764_9403472

In [103]:
# train.describe()

,Click,Weekday,Hour,Timestamp,Log Type,Region,City,Ad Exchange,Ad slot width,Ad slot height,Ad slot visibility,Ad slot format,Ad slot floor price (RMB/CPM),Advertiser ID
count,500000.000000,500000.000000,500000.000000,5.000000e+05,500000,500000.00000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000
mean,0.000726,3.008416,13.788118,2.013061e+16,1,145.27144,150.150642,2.197826,580.683144,183.402060,2.214510,0.277096,29.403108,3386
std,0.026935,2.037453,6.622758,2.021928e+09,0,103.84737,105.052921,0.793436,320.487566,120.010535,19.701588,0.639277,36.812440,0
min,0.000000,0.000000,0.000000,2.013061e+16,1,0.00000,0.000000,1.000000,120.000000,60.000000,0.000000,0.000000,0.000000,3386
25%,0.000000,1.000000,10.000000,2.013061e+16,1,65.00000,75.000000,2.000000,300.000000,90.000000,0.000000,0.000000,4.000000,3386
50%,0.000000,3.000000,15.000000,2.013061e+16,1,146.00000,147.000000,2.000000,468.000000,90.000000,0.000000,0.000000,10.000000,3386
75%,0.000000,5.000000,19.000000,2.013061e+16,1,216.00000,221.000000,3.000000,1000.000000,250.000000,2.000000,0.000000,50.000000,3386
max,1.000000,6.000000,23.000000,2.013061e+16,1,395.00000,399.000000,3.000000,1000.000000,600.000000,255.000000,5.000000,300.000000,3386


In [87]:
### Combine train and test to re-factor the structure and build binary features for logistic regression
print train.iloc[:,1:].shape
print test.shape
all_data = pd.concat([train.iloc[:,1:], test], axis=0)
print(all_data.shape)

(500000, 23)
(545421, 23)
(1045421, 23)


In [88]:
### Feature splitting for combining data
### Test for pd.get_dummies

### Expand Weekdays
weekdays = pd.get_dummies(all_data["Weekday"], prefix = "Weekday")

### Expand Hour
hour = pd.get_dummies(all_data["Hour"], prefix = "Hour")

### Expand OS and Browser
os_and_browser = all_data["User‐Agent"].str.split("_", expand = True)
os = os_and_browser[0]
browser = os_and_browser[1]
os_dummies = pd.get_dummies(os, prefix="OS")
browser_dummies = pd.get_dummies(browser, prefix="Browser")

### Expand floor price
def price_transfer(x):
  if x == 0:
    return '0'
  elif x >= 1 and x <= 10:
    return '1-10'
  elif x >= 11 and x <= 50:
    return '11-50'
  elif x >= 51 and x <= 100:
    return '51-100'
  else:
    return 'larger'

### Directly use dataframe method 'apply' to process series, instead of using map and lambda
floor_price = all_data["Ad slot floor price (RMB/CPM)"].astype(int).apply(price_transfer)
# floor_price = map(lambda price: price_transfer(price), train["Ad slot floor price (RMB/CPM)"].astype(int))

floor_price = pd.get_dummies(floor_price, prefix = "Price")

In [89]:
### Combining features and split all_data to train and test
all_data_dummies = pd.concat([weekdays, hour, os_dummies, browser_dummies, floor_price], axis = 1)
mytrain = all_data_dummies.iloc[:trainlen]
test = all_data_dummies.iloc[trainlen:]
print(mytrain.shape)
mytrain.head()

(500000, 51)


,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Hour_0,Hour_1,Hour_2,...,Browser_opera,Browser_other,Browser_safari,Browser_sogou,Browser_theworld,Price_0,Price_1-10,Price_11-50,Price_51-100,Price_larger
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [90]:
### Read user tags features from csv file
usertagsfile = 'usertags_train.txt'
usertags_train = pd.read_csv(usertagsfile, sep = ' ')
usertagsfile = 'usertags_test.txt'
usertags_test = pd.read_csv(usertagsfile, sep = ' ')

mytrain = pd.concat([mytrain, usertags_train], axis = 1)
test = pd.concat([test, usertags_test], axis = 1)
test.shape

(545421, 118)

In [91]:
### Logistic Regression
X,x,Y,y = train_test_split(mytrain, train["Click"], test_size = 0.1)
     
lr = LogisticRegression(C=0.1)
lr.fit(X, Y)
result = lr.predict_proba(x)

In [92]:
### AUC Evaluation
auc = roc_auc_score(y, result[:,1])
print(auc)

0.668751870795


In [93]:
### MSE Evaluation
mse = mean_squared_error(y, result[:,1])
print(mse)

0.000856288086381


In [94]:
### Make Prediction for test set
preds_lr =  lr.predict_proba(test)[:,1]

def write_prediction(filename, preds):
    submission = open(filename, "w")
    submission.write("Id,Prediction\n")

    # for i in range(10):
    for i in range(len(preds)):
        submission.write("%d,%.5f\n" % (i + 1, preds[i]))

    submission.close()

write_prediction("submission_lr.csv", preds_lr)

In [95]:
### GBRT feature engineering
def transfer(x, dict):
    return dict[x]


counts = all_data["City"].value_counts()
counts = dict(counts)
city = all_data["City"].apply(transfer, args=(counts,))

counts = all_data["Region"].value_counts()
counts = dict(counts)
region = all_data["Region"].apply(transfer, args=(counts,))
# print(city_transfer(1, counts))

In [96]:
counts = os.value_counts()
counts = dict(counts)
os_freq = os.apply(transfer, args=(counts,))
counts = browser.value_counts()
counts = dict(counts)
browser_freq = browser.apply(transfer, args=(counts,))


In [111]:
gbrt_all_data = pd.concat([all_data.iloc[:, 0:2], os_freq, browser_freq, city,region, all_data.iloc[:, 14:19], all_data["Ad slot floor price (RMB/CPM)"]], axis = 1)
gbrt_all_data.head()

,Weekday,Hour,0,1,City,Region,Ad slot width,Ad slot height,Ad slot visibility,Ad slot format,Ad slot floor price (RMB/CPM),Ad slot floor price (RMB/CPM)
0,6,0,1019875,658381,10182,83404,300,250,1,1,0,0
1,6,0,1019875,658381,24669,61313,300,250,0,1,0,0
2,6,0,1019875,658381,672,14266,300,250,2,1,0,0
3,6,0,1019875,658381,31391,126426,468,60,1,0,5,5
4,6,0,1019875,658381,1481,25548,300,250,2,1,0,0


In [112]:
### Split to train and test
gbrt_train = gbrt_all_data.iloc[:trainlen]
gbrt_test = gbrt_all_data.iloc[trainlen:]
print(gbrt_train.shape)
gbrt_test.head()

(500000, 12)


,Weekday,Hour,0,1,City,Region,Ad slot width,Ad slot height,Ad slot visibility,Ad slot format,Ad slot floor price (RMB/CPM),Ad slot floor price (RMB/CPM)
0,1,6,1019875,337416,1161,21820,336,280,0,1,0,0
1,6,0,1019875,658381,17457,32077,300,250,1,0,5,5
2,1,16,1019875,658381,15665,83404,300,250,1,1,0,0
3,6,23,1019875,658381,24669,61313,300,250,2,0,5,5
4,6,20,1019875,337416,53926,53926,960,90,0,0,20,20


In [113]:
### GBRT
X,x,Y,y = train_test_split(gbrt_train, train["Click"], test_size = 0.1)
     
### Based on the Paper "RTB benchmarking with iPingyou dataset" to set attributes
gbrt = GradientBoostingRegressor(n_estimators=50, learning_rate=0.05,max_depth=5).fit(X, Y)
gbrt_result = gbrt.predict(x)

In [115]:
### AUC Evaluation
auc = roc_auc_score(y, gbrt_result)
print(auc)

0.639393818102


In [116]:
### MSE Evaluation
mse = mean_squared_error(y, gbrt_result)
print(mse)

0.000658459644874


In [117]:
### Predict and Write to prediction file
preds_gbrt = gbrt.predict(gbrt_test)
write_prediction("submission_gbrt.csv", preds_gbrt)
# print preds_gbrt